In [ ]:
%%capture
!pip install kaggle_environments;

In [ ]:
utimport pandas as pd
import numpy as np
from kaggle_environments import make, evaluate, utils
import random

In [ ]:
from types import ModuleType
env = make("connectx", debug=True)
env.render(mode="ipython")

In [ ]:
def count_window(grid, player, window_size):

  num_windows = 0

  ## horizontal search
  for row in range(grid.shape[0]):
    for col in range(grid.shape[1] - 3):

      grid_slice = grid[row, col:col+4]

      if np.count_nonzero(grid_slice == player) == window_size:

        if np.all(grid_slice[np.where(grid_slice != player)[0]] == 0):
          num_windows+= 1



  ## vertical search
  for col in range(grid.shape[1]):
    for row in range(grid.shape[0] - 3):
      grid_slice = grid[row:row+4, col]

      if np.count_nonzero(grid_slice == player) == window_size:

         if np.all(grid_slice[np.where(grid_slice != player)[0]] == 0):
          num_windows+= 1


  ## diagonal search
  for row in range(grid.shape[0] - 3):
    for col in range(grid.shape[1] - 3):
      window = np.diagonal(grid[row:row+4, col:col+4])
      if np.count_nonzero(window == player) == window_size:

         if np.all(window[np.where(window != player)[0]] == 0):
            num_windows+= 1

  # negative diagonal search
  for row in range(3, grid.shape[0]):
    for col in range(grid.shape[1] - 3):
      window = np.diagonal(np.fliplr(grid[row-3:row+1, col:col+4]))
      if np.count_nonzero(window == player) == window_size:

        if np.all(window[np.where(window != player)[0]] == 0):
          num_windows+= 1


  return num_windows


In [ ]:
## Function to provide a score on a per column basis provided to
## agent to inform decision making. One Step Lookahead

def score_move(grid, player):



  ## counting windows
  threes = count_window(grid, player, 3)
  fours = count_window(grid, player, 4)
  twos = count_window(grid, player, 2)


  opp_threes = count_window(grid, player%2+1, 3)
  opp_four = count_window(grid, player%2+1, 4)
  opp_twos = count_window(grid, player%2+1, 2)


  score = (fours * 1000) - (opp_threes * 1000) + 100*threes + 1*twos - (opp_twos * 10) - (opp_four*10000)

  return score


In [ ]:
def minimax(grid, col_index, depth, maximizingPlayer, player, configuration):
    # Check if column is valid
    if grid[0,col_index] != 0:
        return np.inf if maximizingPlayer else -np.inf

    # Drop piece in the column
    row = np.where(grid[:, col_index] == 0)[0][-1]
    grid[row,col_index] = player

    # Check for terminal state or max depth
    if count_window(grid, player, configuration.inarow) > 0:
        return 10000 if maximizingPlayer else -10000
    if count_window(grid, player%2+1, configuration.inarow) > 0:
        return -10000 if maximizingPlayer else 10000
    if depth == 0 or np.all(grid != 0):
        return score_move(grid, player%2+1)

    # Recursive minimax calls
    if maximizingPlayer:
        max_score = -np.inf
        for c in range(configuration.columns):
            score = minimax(grid.copy(), c, depth - 1, False, 3 - player, configuration)
            max_score = max(max_score, score)
        return max_score
    else:
        min_score = np.inf
        for c in range(configuration.columns):
            score = minimax(grid.copy(), c, depth - 1, True, 3 - player, configuration)
            min_score = min(min_score, score)
        return min_score




/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


When your agent is being evaluated against others, it will not have access to the Kaggle docker image. Only the following can be imported: Python Standard Library Modules, gym, numpy, scipy, pytorch (1.3.1, cpu only), and more may be added later.

In [ ]:
def my_agent(observation, configuration):

  rows = configuration.rows
  columns = configuration.columns

  inarow = configuration.inarow

  board = observation.board
  player = observation.mark

  ## convert board to 2D grid
  grid = np.asarray(board).reshape(rows, columns)



  ## list of all possible choices
  choices = [c for c in range(columns) if board[c] == 0]

  ## scoring all possible choices - via one step lookahead
  scores = []
  for col_index in choices:
    scores.append(minimax(grid.copy(), col_index, 3, True, observation.mark, configuration))


  ## return best choice
  return choices[np.argmax(scores)]

In [ ]:
trainer = env.train([None, "negamax"])

observation = trainer.reset()

while not env.done:
    my_action = my_agent(observation, env.configuration)
    print("My Action", my_action)
    observation, reward, done, info = trainer.step(my_action)
env.render(mode="ipython")

My Action 0
My Action 1
My Action 3
My Action 1
My Action 1
My Action 2
My Action 2
My Action 3
My Action 6
My Action 5
My Action 3
My Action 0


In [ ]:
def mean_reward(rewards):
    return sum(r[0] for r in rewards) / float(len(rewards))

# Run multiple episodes to estimate its performance.
print("My Agent vs Random Agent:", mean_reward(evaluate("connectx", [my_agent, "random"], num_episodes=5)))
print("My Agent vs Negamax Agent:", mean_reward(evaluate("connectx", [my_agent, "negamax"], num_episodes=5)))

KeyboardInterrupt: 

In [ ]:
import inspect
import os

def write_agent_to_file(function, file):
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write(inspect.getsource(function))
        print(function, "written to", file)

write_agent_to_file(count_window, "submission.py")
write_agent_to_file(score_move, "submission.py")
write_agent_to_file(minimax, "submission.py")
write_agent_to_file(my_agent, "submission.py")

<function count_window at 0x7ae6f4d28a60> written to submission.py
<function score_move at 0x7ae6f4d28430> written to submission.py
<function minimax at 0x7ae6f3443760> written to submission.py
<function my_agent at 0x7ae6f4d2bb50> written to submission.py
